# ITT 13 Spectra - Machine Learning for Natural Language to SQL

## Imports

In [8]:
import json
import numpy as np
import nltk
import shlex

## Helper Functions

In [2]:
def split(text):
    """ Splits a text string into basic tokens """
    if isinstance(text, list):
        # If input is a list of strings then apply recursively to each element in list
        return [split(t) for t in text]
    try:
        return shlex.split(text, posix = False)
    except:
        return ''.join(text)

def stemmer(stemmer, y_tokens):
    """ Wraps a stemming function to not split quotes """
    def _stem(tokens):
        ret = []
        for token in tokens:
            if '"' in token or "'" in token or token in y_tokens:
                ret.append(token)
            else:
                ret.append(stemmer(token))
        return split(' '.join(ret))
    return _stem

def preprocess(data, stem = False):
    """ Returns a list of 2D numpy arrays each corresponding to a text input """
    unique = set()
    for d in data:
        unique |= set(stem(d)) if stem else set(d)
    encode    = {k : v for v, k in enumerate(unique)}
    processed = []
    for d in data:
        observation = []
        tokens      = stem(d) if stem else d
        for token in tokens:
            vector = np.zeros(len(encode) + 1)
            vector[encode[token]] = 1
            observation.append(vector)
        processed.append(np.array(observation))
    return processed, np.array(list(unique) + ['']) # Blank to denote empty string for x or end of query for y

def tidy(X, limit):
    """ To make all data points be the same length by padding it (not neccessary but easier computations) """
    ret = np.zeros((len(X), limit, X[0].shape[1]))
    for i, x in enumerate(X):
        for r, c in zip(*np.where(x == 1)):
            ret[i, r, c] = 1
        for j in range(len(x), limit):
            ret[i, j, -1] = 1
    return ret

## Reading the Data

In [3]:
with open('train_others.json') as js:
    data = json.load(js)
    
# Which data base to train on
# {'academic', 'geo', 'imdb', 'restaurants', 'scholar', 'yelp'}
# [181, 564, 109, 125, 569, 111] (number of data points in each db_id)
db_id = 'geo'

x_q   = [] # question
y_sql = [] # sql

for d in data:
    if d['db_id'] == db_id:
        x_q.append(split(d['question']))
        y_sql.append(split(d['query']))

## Processing the Data

In [4]:
Y, y_tokens = preprocess(y_sql)

stem        = stemmer(nltk.PorterStemmer().stem, y_tokens)

X, x_tokens = preprocess(x_q, stem)

max_len = max(map(len, X))
limit   = (max_len // 10 + 1) * 10 # Pad X such that each observation is max_len rounded up to the nearest 10
X       = tidy(X, limit)

# Check first data point if it has been tokenised and padded appropriately
print(x_q[0])
print(x_tokens[X[0].argmax(axis = 1)].tolist()) # padded version with empty strings

['what', 'is', 'the', 'biggest', 'city', 'in', 'wyoming']
['what', 'is', 'the', 'biggest', 'city', 'in', 'wyom', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
